In [1]:
pip install torchinfo


/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tqdm

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 76 kB 639 kB/s  eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import argparse
import os
import numpy as np


import torch
from PIL import Image
from tqdm import tqdm
from torchvision import transforms

import random

def _compress(tensor, bit):
    max_val = 2**bit - 1
    tensor = torch.clamp(tensor, 0.0, 1.0) * max_val
    tensor = torch.round(tensor)
    tensor = tensor / max_val
    return tensor

def tensor2img(tensor):
    tensor = tensor.cpu()
    tensor = tensor.detach().numpy()
    tensor = np.squeeze(tensor)
    tensor = np.moveaxis(tensor, 0, 2)
    tensor = (tensor * 255)  # + 0.5  # ? add 0.5 to rounding
    tensor = tensor.clip(0, 255).astype(np.uint8)

    img = Image.fromarray(tensor)
    return img

def mkdir(directory, mode=0o777):
    if not os.path.exists(directory):
        os.makedirs(directory)
        os.chmod(directory, mode=mode)

def dir_exists(directory):
    return os.path.exists(directory)

def crop(img_arr, block_size):
    h_b, w_b = block_size
    v_splited = np.vsplit(img_arr, img_arr.shape[0]//h_b)
    h_splited = np.concatenate(
        [np.hsplit(col, img_arr.shape[1]//w_b) for col in v_splited], 0)
    return h_splited

def generate_patches(src_path, files, set_path, crop_size, img_format, max_patches):
    img_path = os.path.join(src_path, files)
    img = Image.open(img_path).convert('RGB')

    name, _ = files.split('.')
    filedir = os.path.join(set_path, 'a')
    if not dir_exists(filedir):
        mkdir(filedir)
        
    filedirb = os.path.join(set_path, 'b')
    if not dir_exists(filedirb):
        mkdir(filedirb)

    img = np.array(img)
    h, w = img.shape[0], img.shape[1]

    if crop_size == None:
        img = np.copy(img)
        img_patches = np.expand_dims(img, 0)
    else:
        rem_h = (h % crop_size[0])
        rem_w = (w % crop_size[1])
        img = img[:h-rem_h, :w-rem_w]
        img_patches = crop(img, crop_size)

    # print('Cropped')

    n = 0

    for i in range(min(len(img_patches), max_patches)):
        img = Image.fromarray(img_patches[i])
        img1 = Image.fromarray(img_patches[i])
        img1 = transforms.ToTensor()(img1)
        img2 = transforms.ToTensor()(img)
        img3 = _compress(img1, 5)
        img_compressed = tensor2img(img3)
        
        
        img_compressed.save(
            os.path.join(filedirb, '{}_{}.{}'.format(name, i, img_format))
        )
        img.save(
            os.path.join(filedir, '{}_{}.{}'.format(name, i, img_format))
        )

        n += 1

    return n


def main(target_dataset_folder, dataset_path, crop_size, img_format, max_patches, max_n):
    print('[ Creating Dataset ]')
    print('Crop Size : {}'.format(crop_size))
    print('Target       : {}'.format(target_dataset_folder))
    print('Dataset       : {}'.format(dataset_path))
    print('Format    : {}'.format(img_format))
    print('Max N    : {}'.format(max_n))

    src_path = dataset_path
    if not dir_exists(src_path):
        raise(RuntimeError('Source folder not found, please put your dataset there'))

    set_path = target_dataset_folder

    mkdir(set_path)

    img_files = os.listdir(src_path)

    max = len(img_files)
    bar = tqdm(img_files)
    i = 0
    j = 0
    for files in bar:
        k = generate_patches(src_path, files, set_path,
                             crop_size, img_format, max_patches)

        bar.set_description(desc='itr: %d/%d' % (
            i, max
        ))

        j += k

        if j >= max_n:
            # Stop the process
            print('Dataset count has been fullfuled')
            break

        i += 1

    print('Dataset Created')

main('dataset/train', '/home/sd/zsq_python/DIV2K_train_HR/', [128, 128], 'PNG', 10, 10000)

[ Creating Dataset ]
Crop Size : [128, 128]
Target       : dataset/train
Dataset       : /home/sd/zsq_python/DIV2K_train_HR/
Format    : PNG
Max N    : 10000


itr: 799/800: 100%|██████████| 800/800 [12:04<00:00,  1.10it/s]

Dataset Created


In [2]:
main('dataset/validation', '/home/sd/zsq_python/DIV2K_valid_HR/', [128, 128], 'PNG', 1, 50)

[ Creating Dataset ]
Crop Size : [128, 128]
Target       : dataset/validation
Dataset       : /home/sd/zsq_python/DIV2K_valid_HR/
Format    : PNG
Max N    : 50


itr: 49/100:  49%|████▉     | 49/100 [00:13<00:14,  3.61it/s]

Dataset count has been fullfuled
Dataset Created


In [3]:
import torch
import os
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim


from torchvision import transforms

def show_tensor(tensor):
    plt.imshow(tensor.permute(1, 2, 0))

class ModelSubtraction(nn.Module):
    def __init__(self, n_layers):
        super(ModelSubtraction, self).__init__()
        
        networks = [
            nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU()
        ]
        
        for i in range(n_layers):
            networks += [
                nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
                nn.ReLU()
            ]
        
        networks += [
            nn.Conv2d(64, 3, kernel_size=5, stride=1, padding=2),
                nn.ReLU()
        ]
        
        self.model = nn.Sequential(*networks)
        
    def forward(self, x):
        y = self.model(x)
        return x - y

class ModelAddition(nn.Module):
    def __init__(self, n_layers):
        super(ModelAddition, self).__init__()
        
        networks = [
            nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU()
        ]
        
        for i in range(n_layers):
            networks += [
                nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
                nn.ReLU()
            ]
        
        networks += [
            nn.Conv2d(64, 3, kernel_size=5, stride=1, padding=2),
                nn.ReLU()
        ]
        
        self.model = nn.Sequential(*networks)
        
    def forward(self, x):
        y = self.model(x)
        return y + x

class Model(nn.Module):
    def __init__(self, n_layers_sub = 6, n_layers_add = 6):
        super(Model, self).__init__()
        
        self.subs = ModelSubtraction(n_layers=n_layers_sub)
        self.add = ModelAddition(n_layers=n_layers_add)
        
        
    def forward(self, x):
        y = self.subs(x)
        y = self.add(y)
        
        return y

import torch.utils.data as data

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in [".png", ".PNG", ".jpg", ".jpeg"])

class DatasetFromFolder(data.Dataset):
    def __init__(self, image_dir):
        super(DatasetFromFolder, self).__init__()
        self.a_path = os.path.join(image_dir, "a")
        self.b_path = os.path.join(image_dir, "b")

        self.image_filenames = [x for x in listdir(
            self.a_path) if is_image_file(x)]

    def __getitem__(self, index):
        a = Image.open(
            os.path.join(self.a_path, self.image_filenames[index])).convert('RGB')
        
        b = Image.open(
            os.path.join(self.b_path, self.image_filenames[index])).convert('RGB')
        
        a = transforms.ToTensor()(a)
        b = transforms.ToTensor()(b)
        
        return a, b

    def __len__(self):
        return len(self.image_filenames)

In [4]:
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm
from os import listdir
import statistics
from torchinfo import summary

def psnr(ground, compressed):
    np_ground = np.array(ground, dtype='float')
    np_compressed = np.array(compressed, dtype='float')
    mse = np.mean((np_ground - np_compressed)**2)
    psnr = np.log10(255**2/mse) * 10
    return psnr

torch.cuda.manual_seed(100)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_set = DatasetFromFolder('dataset/train')
training_data_loader = DataLoader(
        dataset=train_set, num_workers=4, batch_size=4, shuffle=True)
validate_set = DatasetFromFolder('dataset/validation')
validate_data_loader = DataLoader(
        dataset=validate_set, num_workers=4, batch_size=4, shuffle=True)


#image = plt.imread('dataset/train/b/0802x2_3.PNG')

#image_tensor = transforms.ToTensor()(image) # (C, H, W) -> (N, C, H, W)
#image_tensor = image_tensor.unsqueeze(0).to(device)

loss = nn.MSELoss()

net = Model().to(device)

summ = summary(net, input_size=(4, 3, 128, 128))

optimizer = optim.Adam(net.parameters() ,lr=0.001)

epoch = 100

list_psnr = list()
psnr_sum = list()
for i in range(epoch):
    data_len = len(training_data_loader)
    datas = tqdm(enumerate(training_data_loader, 1), total=data_len)
    loss_o_sum = 0
    
    data_len_valid = len(validate_data_loader)
    datas_valid = tqdm(enumerate(validate_data_loader, 1), total=data_len_valid, disable=True)
    
    for iteration, batch in datas:
        image_a, image_b = batch[0].to(device), batch[1].to(device)
        #image = torch.randn((1, 3, 128, 128))
        optimizer.zero_grad() #Membuat gradien model menjadi 0
        output = net(image_b) #forward input ke model
        loss_o = loss(output, image_a) #hitung loss
        loss_o.backward() #menghitung gradien dari model
        
        optimizer.step() #mengoptimasi parameter model
        loss_o_sum += loss_o.item()

        datas.set_description(desc='itr: %d/%d [%3d/%3d] Loss: %.6f' % (
                    iteration, data_len, i, epoch, loss_o_sum/max(1, iteration)
                ))
    for iteration, batch in datas_valid:
        valid_a, valid_b = batch[0].to(device), batch[1].to(device)
        outimg = net(valid_b)
        hasil = tensor2img(outimg[0])
        source = tensor2img(valid_a[0])
        psnr_o = psnr(hasil, source)
        psnr_sum.append(psnr_o)
        
    list_psnr.append(statistics.mean(psnr_sum))
        
torch.save(net, 'model.pth')
print(list_psnr)

itr: 2000/2000 [ 99/100] Loss: 0.000087: 100%|██████████| 2000/2000 [03:07<00:00, 10.68it/s]


[40.83823483630491, 40.62857213441958, 40.6447251965017, 40.65049609190179, 40.61640899277512, 40.765732168951054, 40.93943347407066, 40.89945428499812, 40.896184851667236, 40.821677001026394, 40.838087108170754, 40.897864664176495, 40.89354902476665, 40.98688635541137, 40.950558830053794, 41.007984460122444, 40.97508882106988, 41.02238014549009, 40.99916813764918, 40.96081649348479, 40.950276437106076, 40.93904622971211, 40.91355717622274, 40.948311138036075, 40.98317397615884, 41.02907523440226, 41.068056571818985, 41.05275580288886, 41.02950430828962, 41.00768581917668, 41.01068412999176, 40.98802693105935, 40.981896239459644, 40.9650619953726, 40.96028797991337, 40.99033131316943, 40.97765487222852, 40.968152155449, 40.98565685199935, 40.96843726854176, 40.958072419139484, 40.940363850976, 40.928746765753544, 40.91876469231091, 40.94160485717705, 40.97170940574933, 40.96516905244159, 40.959771483770666, 40.95645016545658, 40.977786197227836, 40.96515826798632, 40.955185158029934, 4

In [5]:
import pandas

!rm -rf dataset

d = {'PSNR': list_psnr}
df = pandas.DataFrame(data=d)
df.to_csv('psnr.csv', index=False)

!zip hasil_training_DRCNN_5bit.zip model.pth psnr.csv

  adding: model.pth (deflated 8%)
  adding: psnr.csv (deflated 55%)


In [6]:
summ

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    --                        --
├─ModelSubtraction: 1-1                  [4, 3, 128, 128]          --
│    └─Sequential: 2-1                   [4, 3, 128, 128]          --
│    │    └─Conv2d: 3-1                  [4, 64, 128, 128]         4,864
│    │    └─ReLU: 3-2                    [4, 64, 128, 128]         --
│    │    └─Conv2d: 3-3                  [4, 64, 128, 128]         36,928
│    │    └─ReLU: 3-4                    [4, 64, 128, 128]         --
│    │    └─Conv2d: 3-5                  [4, 64, 128, 128]         36,928
│    │    └─ReLU: 3-6                    [4, 64, 128, 128]         --
│    │    └─Conv2d: 3-7                  [4, 64, 128, 128]         36,928
│    │    └─ReLU: 3-8                    [4, 64, 128, 128]         --
│    │    └─Conv2d: 3-9                  [4, 64, 128, 128]         36,928
│    │    └─ReLU: 3-10                   [4, 64, 128, 128]        